In [300]:
import cv2
import numpy as np
import os, glob, time
import tensorflow as tf
import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd, string
from ultralytics import YOLO

In [301]:
corners_model = YOLO("detect_corners_n.pt")
info_model = YOLO("detect_info_n.pt")
detect_model = YOLO("digit_detection.pt")
predict_model = tf.keras.models.load_model('digit_recognition.keras', compile=False)


In [302]:
IMG_DIR = "test_1_img"
WARP_DIR = os.path.join(IMG_DIR, "warped")
REGION_ROOT  = os.path.join(IMG_DIR, "regions")
PREVIEW_DIR = os.path.join(IMG_DIR, "preview")

os.makedirs(WARP_DIR, exist_ok=True)
os.makedirs(REGION_ROOT, exist_ok=True)

class_map_corners = {
    2: 'top_left',
    3: 'top_right',
    1: 'bottom_right',
    0: 'bottom_left'
}
class_map_info = {
    2: "name",
    1: "id",
    0: "dob"
}

class_map_digits = {
    0: 'number',
    1: 'slash'
}

In [ ]:
def find_miss_corner(coord):
    # must match this exact order:
    keys = ['top_left','top_right','bottom_left','bottom_right']
    for i,k in enumerate(keys):
        if k not in coord:
            return i
    return -1

def calculate_missed_coord_corner(coord):
    idx = find_miss_corner(coord)
    # 0 → top_left missing
    if idx == 0:
        m = (np.array(coord['top_right']) + np.array(coord['bottom_left'])) / 2
        coord['top_left'] = (2*m - coord['bottom_right']).tolist()
    # 1 → top_right missing
    elif idx == 1:
        m = (np.array(coord['top_left']) + np.array(coord['bottom_right'])) / 2
        coord['top_right'] = (2*m - coord['bottom_left']).tolist()
    # 2 → bottom_left missing
    elif idx == 2:
        m = (np.array(coord['top_left']) + np.array(coord['bottom_right'])) / 2
        coord['bottom_left'] = (2*m - coord['top_right']).tolist()
    # 3 → bottom_right missing
    elif idx == 3:
        m = (np.array(coord['bottom_left']) + np.array(coord['top_right'])) / 2
        coord['bottom_right'] = (2*m - coord['top_left']).tolist()
    return coord

def perspective_transform(image, src_pts):
    dst_pts = np.float32([[0,0],[500,0],[500,300],[0,300]])
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    return cv2.warpPerspective(image, M, (500,300))

def process_image_corner(img_path):
   
    fname = os.path.basename(img_path)
    img_np = cv2.imread(img_path)

    res     = corners_model(img_np, imgsz=640-32, verbose=False)[0]
    boxes   = res.boxes.xyxy.cpu().numpy()
    scores  = res.boxes.conf.cpu().numpy()
    classes = res.boxes.cls.cpu().numpy().astype(int)


    # Select best-confidence box for each corner
    coord_dict = {}
    for cls_id, bbox, conf in zip(classes, boxes, scores):
        corner = class_map_corners.get(int(cls_id))
        if not corner:
            continue
        prev = coord_dict.get(corner)
        if prev is None or conf > prev[1]:
            coord_dict[corner] = (bbox, conf)

    # Calculate center points
    centers = {
        name: ((b[0]+b[2])/2, (b[1]+b[3])/2)
        for name,(b,_) in coord_dict.items()
    }

    if len(centers) < 3:
        #print(f"[{fname}] only {len(centers)}/4 corners — saving original unchanged")
        cv2.imwrite(os.path.join(WARP_DIR, fname), img_np)
        return
    if len(centers) == 3:
        centers = calculate_missed_coord_corner(centers)

    # Warp and save
    src = np.float32([
        centers['top_left'],
        centers['top_right'],
        centers['bottom_right'],
        centers['bottom_left']
    ])
    warp = perspective_transform(img_np, src)
    out_name = os.path.splitext(fname)[0] + ".jpg"
    out_path = os.path.join(WARP_DIR, out_name)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    cv2.imwrite(out_path, warp)


def process_image_infor(img_path):
    fname = os.path.basename(img_path)
    base, ext = os.path.splitext(fname)

    # 1) Run inference
    res     = info_model(img_path, imgsz=480, conf=0.25, verbose=False)[0]
    boxes   = res.boxes.xyxy.cpu().numpy()
    classes = res.boxes.cls.cpu().numpy().astype(int)

    # 2) For each detection, crop and save
    img = cv2.imread(img_path)
    for box, cls_id in zip(boxes, classes):
        field = class_map_info.get(int(cls_id))
        if field is None:
            continue

        x1, y1, x2, y2 = box.astype(int)
        crop = img[y1:y2, x1:x2]

        out_name = f"{base}.jpg"
        out_path = os.path.join(REGION_ROOT, field, out_name)
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        cv2.imwrite(out_path, crop)

def cropped_digits(img_path, field):
    if field == "dob":
        output_dir = os.path.join(REGION_ROOT, "digits_dob")
    else:
        output_dir = os.path.join(REGION_ROOT, "digits_id")
    fname = os.path.basename(img_path)
    base, ext = os.path.splitext(fname)
    
    # 1) Run inference
    res = detect_model(img_path, conf=0.85, verbose=False, save=True, save_txt=True)[0]
    boxes   = res.boxes.xyxy.cpu().numpy()
    classes = res.boxes.cls.cpu().numpy().astype(int)
    img = cv2.imread(img_path)
 
    bboxes = []
    for cls_id, (x1, y1, x2, y2) in zip(classes, boxes):
        if cls_id == 0:
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            if x2 - x1 > 2 and y2 - y1 > 2:  # filter tiny boxes
                bboxes.append((x1, y1, x2, y2))

    if not bboxes:
        #print(f"[{fname}] No class-0 digits found.")
        return

    # 4) Sort left to right
    bboxes.sort(key=lambda box: box[0])

    # 5) Crop and save
    os.makedirs(output_dir, exist_ok=True)
    for i, (x1, y1, x2, y2) in enumerate(bboxes):
        crop = img[y1:y2, x1:x2]
        crop = cv2.resize(crop, (28, 28))
        out_name = f"{base}_{i:02d}.jpg"
        out_path = os.path.join(output_dir, out_name)
        cv2.imwrite(out_path, crop)

    #print(f"[{fname}] Cropped {len(bboxes)} digits to {output_dir}")

def predict_digits(img_path):
    fname = os.path.basename(img_path)
    base, ext = os.path.splitext(fname)
    # 1) Read all cropped digits
    for field in ["id", "dob"]:
        digit_dir = os.path.join(REGION_ROOT, f"digits_{field}")
        digit_files = sorted(glob.glob(os.path.join(digit_dir, f"{base}_*.jpg")))

        if not digit_files:
            #print(f"[{fname}] No cropped digits found.")
            return
        batch = []
        for file in digit_files:
            img = cv2.imread(file, cv2.IMREAD_COLOR)         
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
            img = cv2.resize(img, (28, 28))
            img = img.astype("float32") 
            batch.append(img)

        batch_input = np.stack(batch, axis=0)

        preds = predict_model.predict(batch_input, verbose=0)
        digits = np.argmax(preds, axis=1)

        digit_str = ''.join(map(str, digits))
        if field == "dob":
            digit_str = f"{digit_str[:2]}/{digit_str[2:4]}/{digit_str[4:8]}{digit_str[8:]}" if len(digit_str) >= 8 else digit_str
        print(f"[{fname}] Predicted {field} digits: {digit_str}")
    return digit_str


In [341]:
def predict():
    for img_path in sorted(glob.glob(os.path.join(IMG_DIR, "*.*"))):
        fname = os.path.basename(img_path)

        #detect 4 corners and save warped images
        t1 = time.perf_counter()
        process_image_corner(img_path)
        print(f"✅ Corners & warp done in {time.perf_counter() - t1:.2f} sec")

        #detect info and save regions
        t2 = time.perf_counter()
        warp_image_path = os.path.join(WARP_DIR, fname )
        process_image_infor(warp_image_path)
        print(f"✅ Info detection done in {time.perf_counter() - t2:.2f} sec")

        #detect digits and save regions
        t3 = time.perf_counter()
        cropped_digits(os.path.join(REGION_ROOT, "dob", fname), "dob")
        cropped_digits(os.path.join(REGION_ROOT, "id", fname), "id")
        
        print(f"✅ Cropped digits done in {time.perf_counter() - t3:.2f} sec")
        #predict digits
        t4 = time.perf_counter()
        predict_digits(img_path)

        print(f"✅ Digit predictions done in {time.perf_counter() - t4:.2f} sec")

In [402]:
predict()

✅ Corners & warp done in 0.06 sec
✅ Info detection done in 0.03 sec
Results saved to runs/detect/predict3
1 label saved to runs/detect/predict3/labels
Results saved to runs/detect/predict3
1 label saved to runs/detect/predict3/labels
✅ Cropped digits done in 0.05 sec
[29.jpg] Predicted id digits: 096202006939
[29.jpg] Predicted dob digits: 01/01/2002
✅ Digit predictions done in 0.08 sec
